In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm, zscore
from scipy.optimize import brentq
from scipy.interpolate import SmoothBivariateSpline

#Black-Scholes model + implied volatility
def black_scholes_call(S, K, T, r, sigma):
    if T <= 1e-6: return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def implied_volatility(price, S, K, T, r):
    if price <= max(S - K * np.exp(-r * T), 0) or price >= S: return np.nan
    try:
        return brentq(lambda sig: black_scholes_call(S, K, T, r, sig) - price, 1e-6, 4.0)
    except: return np.nan

#Data acquisition
ticker_symbol = "NVDA"  # Try "TSLA" or "NVDA" for more "rugged" surfaces
ticker = yf.Ticker(ticker_symbol)
S_spot = ticker.history(period="1d")['Close'].iloc[-1]
r_rate = 0.035 

expirations = ticker.options[1:10]
raw_data = []

print(f"Calibrating Local Vol for {ticker_symbol} | Spot: {S_spot:.2f}")

for exp in expirations:
    chain = ticker.option_chain(exp)
    # Indices need a wider strike range to see the skew
    mult = 0.15 if len(ticker_symbol) <= 3 else 0.10
    calls = chain.calls[(chain.calls['strike'] > S_spot * (1-mult)) & (chain.calls['strike'] < S_spot * (1+mult))]
    T_years = (pd.to_datetime(exp) - pd.Timestamp.now()).days / 365.0
    if T_years <= 0.05: continue
    
    for _, row in calls.iterrows():
        iv = implied_volatility(row['lastPrice'], S_spot, row['strike'], T_years, r_rate)
        if not np.isnan(iv):
            raw_data.append([T_years, row['strike'], iv])

df = pd.DataFrame(raw_data, columns=['T', 'K', 'IV'])

if df.empty:
    raise ValueError("No valid option data found. Check ticker or market hours.")

# Converting to numpy to avoid the Scipy 'AttributeError'
iv_array = df['IV'].to_numpy()
df = df[np.abs(zscore(iv_array)) < 2.0]

pts = df[['T', 'K']].values
vars_list = (df['IV']**2 * df['T']).values
# s=5.0 provides a good balance between SPY smoothness and GOOGL detail
interp_vol_surf = SmoothBivariateSpline(pts[:, 0], pts[:, 1], vars_list, s=0.1)

#Dupire formula
def get_clean_vol(K, T):
    dk, dt = K * 0.005, 0.001
    def p(strike, time):
        # Access spline value correctly from 2D array result
        v = max(interp_vol_surf(time, strike)[0][0], 1e-6)
        return black_scholes_call(S_spot, strike, time, r_rate, np.sqrt(v/time))
    
    C = p(K, T)
    dCdT = (p(K, T + dt) - C) / dt
    dCdK = (p(K + dk, T) - p(K - dk, T)) / (2 * dk)
    d2CdK2 = (p(K + dk, T) - 2*C + p(K - dk, T)) / (dk**2)
    
    num, den = dCdT + r_rate * K * dCdK, 0.5 * K**2 * d2CdK2
    
    # Check for no-arbitrage violations
    if den <= 1e-8 or num <= 0: return np.nan
    return np.sqrt(num / den)

#Grid + auto zoom
T_grid = np.linspace(df['T'].min(), df['T'].max() - 0.01, 50)
K_grid = np.linspace(df['K'].min(), df['K'].max(), 50)
T_mesh, K_mesh = np.meshgrid(T_grid, K_grid)
Z_loc = np.vectorize(get_clean_vol)(K_mesh, T_mesh)

# Statistics for Auto-Zooming the z-axis (sigma)
valid_z = Z_loc[~np.isnan(Z_loc)]
z_min, z_max = np.percentile(valid_z, 2), np.percentile(valid_z, 98)
Z_loc = np.nan_to_num(Z_loc, nan=np.nanmean(valid_z))

#Fancy dashboard
fig = go.Figure(data=[go.Surface(
    z=Z_loc, x=T_grid, y=K_grid, 
    colorscale='Viridis',
    contours_z=dict(show=True, usecolormap=True, highlightcolor="white", project_z=True),
    hovertemplate='Maturity (T): %{x:.2f}<br>Strike (K): %{y:.2f}<br>Local Vol (σ): %{z:.2%}<extra></extra>'
)])

fig.update_layout(
    title=f"Local Volatility Surface: {ticker_symbol} (Spot: {S_spot:.2f})",
    template="plotly_dark",
    scene=dict(
        xaxis_title='Maturity (T)',
        yaxis_title='Strike (K)',
        zaxis_title='Local Vol (σ)',
        zaxis=dict(range=[z_min * 0.8, z_max * 1.2]), # Zooming into the "Plateau"
        camera=dict(eye=dict(x=1.7, y=1.7, z=1.0))
    ),
    width=1100, height=800,
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

def verify_calibration(sample_idx=10):
    #Choose a test option in real data 
    test_option = df.iloc[sample_idx]
    K_test = test_option['K']
    T_test = test_option['T']
    market_iv = test_option['IV']
    
    #Get smooth variance from spline at this point
    #Recall : we interpolayed (IV^2 * T)
    fitted_var_total = interp_vol_surf(T_test, K_test)[0][0]
    fitted_iv = np.sqrt(fitted_var_total / T_test)
    
    #Compute prices
    price_market = black_scholes_call(S_spot, K_test, T_test, r_rate, market_iv)
    price_model = black_scholes_call(S_spot, K_test, T_test, r_rate, fitted_iv)
    
    print(f"\n--- BACKTESTING VERIFICATION ---")
    print(f"Option: Strike {K_test:.2f} | Maturité {T_test:.2f} ans")
    print(f"IV Marché: {market_iv:.2%}")
    print(f"IV lissée (Modèle): {fitted_iv:.2%}")
    print(f"Prix Marché: ${price_market:.4f}")
    print(f"Prix Modèle: ${price_model:.4f}")
    print(f"Erreur Relative: {abs(price_model - price_market)/price_market:.2%}")

verify_calibration()

Calibrating Local Vol for NVDA | Spot: 187.90



--- BACKTESTING VERIFICATION ---
Option: Strike 185.00 | Maturité 0.07 ans
IV Marché: 52.14%
IV lissée (Modèle): 50.80%
Prix Marché: $12.3000
Prix Modèle: $12.0339
Erreur Relative: 2.16%
